In [11]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np

In [12]:
wget="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [13]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(wget)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [19]:
spacex_df['Flight Number']

0      1
1      2
2      3
3      4
4      5
5      7
6      8
7      9
8     10
9     11
10    12
11    13
12    14
13    15
14    16
15    17
16    18
17    19
18    20
19    22
20    23
21    24
22    25
23    26
24    27
25    28
26     6
27    21
28    29
29    37
30    40
31    42
32    46
33    49
34    51
35    55
36    30
37    31
38    32
39    33
40    34
41    35
42    36
43    38
44    39
45    41
46    43
47    44
48    54
49    45
50    47
51    48
52    50
53    52
54    53
55    56
Name: Flight Number, dtype: int64

In [14]:
spacex_df.groupby(by='Launch Site')[['class']].count()

,class
Launch Site,
CCAFS LC-40,26
CCAFS SLC-40,7
KSC LC-39A,13
VAFB SLC-4E,10


In [15]:
launch_sites=spacex_df['Launch Site'].unique()
launch_sites=np.append(launch_sites,'All sites')

In [16]:
success_site=spacex_df.groupby(by='Launch Site')[['class']].sum()

In [17]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                            html.Div([html.Label("Select a launch site:"),
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[{'label':sitio, 'value':sitio} for sitio in launch_sites],
                                    value='All sites'
                                    #, placeholder='Select a report type'
                                )
                                     ]),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"), 
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(min_payload, max_payload, value=[5000, 7000], id='payload-slider')),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')])
def pie_chart(chosen_site):
    if chosen_site == 'All sites':
        figura=px.pie(success_site, values='class', names=success_site.index,
                                       title='Number of successful retrievals by launch site')
        figura.update_traces(textinfo='value')
        return figura
    else:
        specific_site=spacex_df[spacex_df['Launch Site']==chosen_site]
        figura=px.pie(values=specific_site['class'].value_counts().sort_index().values, names=['Failure','Success'],
                                       title='Percentage of successful retrievals in {}'.format(chosen_site))
        return figura
        
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='payload-slider', component_property='value')])
def scatter_chart(chosen_range):
    specific_range=spacex_df[(spacex_df['Payload Mass (kg)']>=chosen_range[0])  & (spacex_df['Payload Mass (kg)']<=chosen_range[1])]
    specific_range['class'].replace(0,'Failure',inplace=True)
    specific_range['class'].replace(1,'Success',inplace=True)
    figura=px.scatter(specific_range, x='Payload Mass (kg)', y='class', title='Percentage of successful retrievals in')
    return figura        
# Run the app
if __name__ == '__main__':
    app.run_server()

C:\Users\Omaro\AppData\Local\Temp\ipykernel_17644\858795416.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Omaro\AppData\Local\Temp\ipykernel_17644\858795416.py:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Omaro\AppData\Local\Temp\ipykernel_17644\858795416.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Omaro\AppData\Local\Temp\ipykernel_17644\858795416.py:58: SettingWithCopyWarning:


A value